In [1]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 50000/50000 [00:02<00:00, 16983.77 examples/s]


{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [3]:
from transformers import AutoTokenizer, DataCollatorForLanguageModeling

In [2]:


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 11.6kB/s]
config.json: 100%|██████████| 570/570 [00:00<00:00, 123kB/s]
vocab.txt: 100%|██████████| 213k/213k [00:00<00:00, 1.62MB/s]
tokenizer.json: 100%|██████████| 436k/436k [00:00<00:00, 2.19MB/s]
Map: 100%|██████████| 50000/50000 [00:09<00:00, 5494.53 examples/s]


In [3]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [4]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

model.safetensors: 100%|██████████| 436M/436M [00:13<00:00, 32.9MB/s] 
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [13]:
import numpy as np
# !pip install evaluate
import evaluate

metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [16]:
# next(iter(small_train_dataset))

{'label': 4,
 'text': "I stalk this truck.  I've been to industrial parks where I pretend to be a tech worker standing in line, strip mall parking lots, and of course the farmer's market.  The bowls are so so absolutely divine.  The owner is super friendly and he makes each bowl by hand with an incredible amount of pride.  You gotta eat here guys!!!",
 'input_ids': [101,
  146,
  27438,
  1142,
  4202,
  119,
  146,
  112,
  1396,
  1151,
  1106,
  3924,
  8412,
  1187,
  146,
  9981,
  1106,
  1129,
  170,
  13395,
  7589,
  2288,
  1107,
  1413,
  117,
  6322,
  8796,
  5030,
  7424,
  117,
  1105,
  1104,
  1736,
  1103,
  9230,
  112,
  188,
  2319,
  119,
  1109,
  20400,
  1132,
  1177,
  1177,
  7284,
  10455,
  119,
  1109,
  3172,
  1110,
  7688,
  4931,
  1105,
  1119,
  2228,
  1296,
  7329,
  1118,
  1289,
  1114,
  1126,
  10965,
  2971,
  1104,
  8188,
  119,
  1192,
  13224,
  3940,
  1303,
  3713,
  106,
  106,
  106,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [11]:
trainer.train()

/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yil115 (abcd_ml). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.567306,0.274000
2,No log,1.168992,0.495000
3,No log,1.047254,0.552000


/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=96, training_loss=1.3363811175028484, metrics={'train_runtime': 44.1768, 'train_samples_per_second': 67.909, 'train_steps_per_second': 2.173, 'total_flos': 789354427392000.0, 'train_loss': 1.3363811175028484, 'epoch': 3.0})

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorForLanguageModeling, LlamaForCausalLM
from datasets import load_dataset

# Replace 'your-model-name' with the actual model you want to use
model_name = "lmsys/vicuna-7b-v1.5"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name).to('cuda:1')

/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.63s/it]


In [2]:


# Load and preprocess the dataset
# Replace 'cerebras/SlimPajama-627B' with the correct dataset path or name
# dataset = load_dataset("cerebras/SlimPajama-627B", streaming=True)

# dataset = load_dataset("DKYoon/SlimPajama-6B", streaming=True)

dataset = load_dataset("eli5", streaming=True)

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=1024)

tokenized_dataset = dataset.map(preprocess_function, batched=True)




FileNotFoundError: https://files.pushshift.io/reddit/submissions

In [7]:
# next(iter(tokenized_dataset['train']))

{'text': 'Want Tori to Coach You Too?\nTori\'s Health Step by Step coming soon.\nWin free copies, prizes, access to exclusive behind-the-scenes, free access to Coach Tori, and more.\nand receive a copy of Tori\'s Weekly Challenges. We\'ll also notify you of when Tori\'s Program becomes available.\nI\'ve been asked, even criticized, about adding a focus on nutrition to Desert. There\'s a reason why. I had poor nutritional examples growing up. Being confused on the issue of nutrition cost me a lot. I remember yo-yo\'ing a lot. The only time I even came close to being my desired weight was when I did high-intensity workouts daily. At one point, I was exercised about 6 hours a day. I was in multiple dance classes and a karate class, as well as another karate club that met for two hours three days a week. I also rode my bike to campus, and even added a one hour workout when I got home. I was still thirty pounds overweight. I can attest to the coined phrase "You cannot exercise away a bad di

In [11]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",           # output directory
    num_train_epochs=3,               # total number of training epochs
    per_device_train_batch_size=1,    # batch size per device during training
    per_device_eval_batch_size=1,     # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir="./logs",             # directory for storing logs
    logging_steps=10,
    max_steps=1000,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    # data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Train the model
trainer.train()

OutOfMemoryError: Caught OutOfMemoryError in replica 3 on device 3.
Original Traceback (most recent call last):
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/torch/nn/parallel/parallel_apply.py", line 64, in _worker
    output = module(*input, **kwargs)
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/transformers/models/llama/modeling_llama.py", line 806, in forward
    outputs = self.model(
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1194, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-packages/transformers/models/llama/modeling_llama.py", line 693, in forward
    layer_outputs = decoder_layer(
  File "/home/shawn/anaconda3/envs/lovim/lib/python3.9/site-

In [4]:
import torch
torch.cuda.empty_cache()
